Les acteurs ont en moyenne quel âge ? données films et acteurs (champs age, birth year et start year)
***age moyen à date du film

1- Appel des deux tables - trie ordre croissant - reset index et drop colonne en trop pour les deux tables

2- Nettoyage de la base Movies :
         - Conservation uniquement des films titleType = movie
         - Retrait des films isadult
         - Suppresion colonne quine me sont plus utiles
         - Explode genre en plusieurs colonnes

3- Nettoyage de la base Actors :
        - Supprimer les acteurs nés avant 1800 pour alléger la base
        - Explode colonne primaryProfession et conserver actor actress
        - Retirer la colonne deathYear
        - Explode colonne knownForTitles en film 1 film 2 etc....
        
        


3- Joindre les tables et retrier :
    - Supprimer les films plus anciens que 1918
    - Suprimer les lignes ou l'âge de l'acteur n'est pas renseigné
(ce tri n'a pas été fait avant car cela permet de nattoyer la base acteurs/movies en même temps)

4 - Créer une colonne âge acteur dans le film : Date du film - date de naissance (startYear - birthYear)

5 - il me faudra une moyenne de cette colonne pour répondre à la question par année






In [165]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
import plotly.express as px 
import plotly.graph_objects as go 
import matplotlib.pyplot as plt
import seaborn as sns

## 1- Appel des trois tables - trie - reset index et drop colonne en trop pour les deux tables :
 actors = name.basics
 movies = title.basics
 title = 

## Table Actors

In [166]:
# appel de la table acteurs

actors = pd.read_csv('https://datasets.imdbws.com/name.basics.tsv.gz', sep = '\t', low_memory=False)

In [167]:
# tri des acteurs par date de naissance 
actors.sort_values(by=['birthYear'], inplace=True)

#reset index 
actors.reset_index(inplace=True)

#drop colonne index en double 
actors.drop('index',axis = 1, inplace=True)



In [168]:
# check si l'ordre est bien modifié et l'index modifié

actors.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0784172,Lucio Anneo Seneca,0004,0065,writer,"tt0218822,tt0049203,tt0237666,tt0972562"
1,nm4511421,Megan Liz Smith,0012,\N,"miscellaneous,actress","tt1966378,tt10183848,tt3416744"
2,nm8810146,Paul Walsh,0021,\N,actor,"tt0314567,tt0093223,tt0098878"
3,nm0430769,Flavius Josephus,0037,0095,"writer,miscellaneous","tt0009573,tt0237666,tt0049833,tt0273236"
4,nm1306202,Plutarch,0046,0122,writer,"tt0056937,tt0272979,tt0237666,tt0472473"


In [169]:
actors.shape # table brute

(11368358, 6)

## Table Movies

In [170]:
# appel de la table films (j'ai pris title basics qui contient l'année du film)

movies = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', sep = '\t', low_memory=False)
movies.sort_values('startYear', inplace=True)

In [171]:
#reset index base movies
movies.reset_index(inplace=True)

#drop colonne index en double 
movies.drop('index',axis = 1, inplace=True)

In [172]:
# check si l'ordre est bien modifié et l'index modifié
movies.head() 

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt3155794,short,Passage de Venus,Passage de Venus,0,1874,\N,1,"Documentary,Short"
1,tt14495706,short,La Rosace Magique,La Rosace Magique,0,1877,\N,1,"Animation,Short"
2,tt12592084,short,Le singe musicien,Le singe musicien,0,1878,\N,1,"Animation,Short"
3,tt2221420,short,Sallie Gardner at a Gallop,Sallie Gardner at a Gallop,0,1878,\N,1,"Documentary,Short"
4,tt15320514,short,Skeleton of Horse,Skeleton of Horse,0,1881,\N,1,"Animation,Short"


In [173]:
movies.shape # table brute

(8479240, 9)

## Table Title

In [174]:
# appel de la table title
title = pd.read_csv("https://datasets.imdbws.com/title.principals.tsv.gz", sep="\t")

In [175]:
# filtre par category actor and actress
title = title.loc[(title['category']=='actor')|(title['category']=='actress')]

In [176]:
#reset index base title
title.reset_index(inplace=True)

#drop colonne index en double 
title.drop('index',axis = 1, inplace=True)

In [177]:
# remplacer les \n par des nan
title.replace('\\N', np.nan, inplace=True)

In [178]:
# check si l'index est bien modifié
title.head()

,tconst,ordering,nconst,category,job,characters
0,tt0000005,1,nm0443482,actor,NaN,"[""Blacksmith""]"
1,tt0000005,2,nm0653042,actor,NaN,"[""Assistant""]"
2,tt0000007,1,nm0179163,actor,NaN,NaN
3,tt0000007,2,nm0183947,actor,NaN,NaN
4,tt0000008,1,nm0653028,actor,NaN,"[""Sneezing Man""]"


## 2-Nettoyage de la base Movies :
###          - Remplacer les \N par des nan
###          - Conservation uniquement des films titleType = movie
###          - Retrait des films isadult
###          - Suppresion des colonnes qui ne me sont plus utiles
###          - suppression des lignes aux données startYear et runtimeMinutes avec des nan
###          - change str en int sur les colonnes startYear et runtimeMinutes et tri sur durée + reset index
###          - Tri des valeurs par nconst + set index

In [179]:
# Remplacer les \N par des nan

movies.replace('\\N', np.nan, inplace=True)

In [180]:
#Conservation uniquement des films titleType = movie

movies = movies[movies['titleType'] == 'movie']

In [181]:
# Retrait des films isadult 

movies = movies[movies['isAdult'] == '0']

In [182]:
# supression des colonnes inutiles
movies = movies.drop(columns=['primaryTitle','titleType', 'endYear', 'isAdult'])

#movies = movies.drop(columns=['primaryTitle','originalTitle','titleType', 'endYear', 'isAdult', 'genres'])

In [183]:
movies.head(2)

,tconst,originalTitle,startYear,runtimeMinutes,genres
682,tt2210499,Birmingham,1896,61,Documentary
1488,tt0229676,Reproduction of the Corbett and Fitzsimmons Fight,1897,NaN,"Documentary,News,Sport"


In [184]:
# suppression des lignes aux données startYear et runtimeMinutes avec des nan

movies.dropna(subset=['startYear', 'runtimeMinutes' ], inplace=True)



In [185]:
# change str en int sur les colonnes startYear et runtimeMinutes
movies = movies.astype({'startYear': int})
movies = movies.astype({'runtimeMinutes': int})

In [186]:
#je ne conserve que les durées entre 58mn et inf à 270mn
movies = movies[(movies['runtimeMinutes']>=58) & (movies['runtimeMinutes']<=270)]

In [187]:
#reset index base movies (again !)
movies.reset_index(inplace=True)

#drop colonne index en double 
movies.drop('index',axis = 1, inplace=True)

In [188]:
# trie par valeur nconst
movies = movies.sort_values('tconst')

#set_index = nconst
movies = movies.set_index('tconst')

In [189]:
#check
movies.head(2)

,originalTitle,startYear,runtimeMinutes,genres
tconst,,,,
tt0000502,Bohemios,1905,100,NaN
tt0000574,The Story of the Kelly Gang,1906,70,"Action,Adventure,Biography"


In [190]:
# table uniquement avec les données conservées
movies.shape 

(326163, 4)

## 3- Nettoyage de la base Actors :
###        - Remplacer les \N par des nan
###        - Supprimer les lignes qui contiennent des nan dans la colonne birthYear  
###        - Retirer les colonnes qui ne me sont pas utiles
###        - Supprimer les acteurs nés avant 1800 + reset index
###        - change str en int sur les colonnes startYear et runtimeMinutes
###        - Tri des valeurs par nconst + set index

In [191]:
# remplacer les \N pas des nan 

actors.replace('\\N', np.nan, inplace=True)

In [192]:
# suppression des lignes qui comportent des Nan dans la colonne birthYear

actors.dropna(subset=['birthYear'], inplace=True)

In [193]:
# Retirer les colonnes qui ne me sont pas utiles
#actors.drop(columns=['deathYear', 'primaryProfession', 'knownForTitles'], inplace=True)

In [194]:
# ne conserve que les rows des acteurs/actrices nés avant 1800
actors = actors[actors['birthYear'] >= '1800']

#reset index base actors (again !)
actors.reset_index(inplace=True)

#drop colonne index en double (again !)
actors.drop('index',axis = 1, inplace=True)

In [195]:
# change str en int sur les colonnes startYear et runtimeMinutes
actors = actors.astype({'birthYear': int})

In [196]:
# Table actors - trie par valeur nconst
actors = actors.sort_values('nconst')

#set_index = nconst
actors = actors.set_index('nconst')

In [197]:
actors.info()

<class 'pandas.core.frame.DataFrame'>
Index: 536849 entries, nm0000001 to nm9993675
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   primaryName        536849 non-null  object
 1   birthYear          536849 non-null  int32 
 2   deathYear          183622 non-null  object
 3   primaryProfession  470277 non-null  object
 4   knownForTitles     523878 non-null  object
dtypes: int32(1), object(4)
memory usage: 22.5+ MB


In [198]:
# check index
actors.head()

,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
nconst,,,,,
nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0050419,tt0053137,tt0072308"
nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0071877,tt0117057,tt0038355,tt0037382"
nm0000003,Brigitte Bardot,1934,NaN,"actress,soundtrack,music_department","tt0056404,tt0049189,tt0054452,tt0057345"
nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0078723,tt0080455,tt0072562,tt0077975"
nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0083922,tt0060827,tt0050986,tt0050976"


In [199]:
# explode colonne primaryProfession
#actors[['firstProfession', 'secondProfession', 'thirdProfession']] = actors['primaryProfession'].str.split(',', n=2, expand=True)

#drop colonne primaryProfession
#actors.drop(columns=['primaryProfession'],axis = 1, inplace=True)

In [200]:
# check dataframe actors
#actors.head()

In [201]:
# Je ne garde que les acteurs/actrices de mes trois colonnes : firstProfession, secondProfession, thirdProfession

#actors = actors[(actors['firstProfession'].isin(['actress','actor'])) | ( actors['secondProfession'].isin(['actress','actor'])) | ( actors['thirdProfession'].isin(['actress','actor']))]

In [202]:
#drop colums firstProfession, secondProfession, thirdProfession qui ne me sont plus utiles
#actors.drop(columns=['firstProfession','secondProfession', 'thirdProfession'],axis = 1, inplace=True)

## 3- Nettoyage de la base Title :
###        - Remplacer les \N par des nan
###        - dropna sur les cellules tconst non renseignées
###        - supprimer les colonnes inutiles
###        - Tri des valeurs par nconst + set index

In [203]:
# Remplacer les \N par des nan

title.replace('\\N', np.nan, inplace=True)

In [204]:
#dropna sur les cellules startyear non renseignées

title.dropna(subset=['tconst'],inplace=True)

In [205]:
title.head(2)

,tconst,ordering,nconst,category,job,characters
0,tt0000005,1,nm0443482,actor,NaN,"[""Blacksmith""]"
1,tt0000005,2,nm0653042,actor,NaN,"[""Assistant""]"


In [206]:
# suppression des colonnes inutiles

title.drop(columns=['ordering','job', 'characters'], inplace=True)

In [207]:
# trie par valeur nconst
title = title.sort_values('nconst')

#set_index = nconst
title = title.set_index('nconst')

In [208]:
title.head()

,tconst,category
nconst,,
nm0000001,tt0055307,actor
nm0000001,tt0031983,actor
nm0000001,tt0029971,actor
nm0000001,tt0507706,actor
nm0000001,tt0038370,actor


## 3- Joindre les tables et retrier :

In [209]:
# Table actors
actors.head(2)


,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
nconst,,,,,
nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0050419,tt0053137,tt0072308"
nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0071877,tt0117057,tt0038355,tt0037382"


In [210]:
# table title:
title.head(2)

,tconst,category
nconst,,
nm0000001,tt0055307,actor
nm0000001,tt0031983,actor


In [211]:
# merge des tables actors et title 
df_actors_title = actors.merge(title , how='inner', on='nconst')

#reset index de la nouvelle table : df_actors_title
df_actors_title = df_actors_title.reset_index()

#check
df_actors_title.head(2)

In [215]:
#set index de la table merge avec la colonne tconst (pour mon prochain merge)
df_actors_title = df_actors_title.set_index('tconst')

#check
df_actors_title.head(2)

In [217]:
# Table  movies
movies.head(2)

,originalTitle,startYear,runtimeMinutes,genres
tconst,,,,
tt0000502,Bohemios,1905,100,NaN
tt0000574,The Story of the Kelly Gang,1906,70,"Action,Adventure,Biography"


In [248]:
# merge des tables df_actors_title  et movies

actors_title_movies = df_actors_title.merge(movies, how='inner', on='tconst' )

# check merge actors_title  et movies
actors_title_movies.head(2)


## Création colonne de l'âge de l'acteur au moment du film

In [282]:
# age_actor_actress = startYear - birthYear
actors_title_movies['age_actor_actress'] = actors_title_movies['startYear'] - actors_title_movies['birthYear']

#check
actors_title_movies.head(2)


,primaryName,birthYear,deathYear,category,originalTitle,startYear,age_actor_actress
0,Fred Astaire,1899,1987,actor,The Pleasure of His Company,1961,62
1,Debbie Reynolds,1932,2016,actress,The Pleasure of His Company,1961,29


## Clean df merge comprenant les 3 tables : actors_title_movies

In [ ]:
#reset index de la table pour que la Pk soit unique
actors_title_movies = actors_title_movies.reset_index()

#drop colonnes qui m'ont servi uniquement à vérifier mes merges
actors_title_movies.drop(columns=['tconst', 'nconst', 'primaryProfession','knownForTitles','runtimeMinutes', 'genres'], inplace=True)

# je tri ici car cela permet de trier toutes les tables d'un seul et éviter de faire 3 fois la même opé
actors_title_movies = actors_title_movies[(actors_title_movies['startYear'] >= 1918) & (actors_title_movies['startYear'] <= 2021)]

#check
actors_title_movies.head()

## Groupby startYear et moyenne des ages des acteurs (tous genres confondus)
### Pour avoir l'âge moyen des acteurs par année

In [347]:
# calcul moyenne age groupby(by=["originalTitle" & "startYear"])
age_mean_year = round(actors_title_movies.groupby(by=["startYear"]).mean('age_actor_actress'),2)

#reset index
age_mean_year = age_mean_year.reset_index()

#drop colonnes
age_mean_year.drop(columns=['index', 'birthYear'], inplace =True)

#rename nom colonne age_actor_actress
age_mean_year.rename(columns={'age_actor_actress':'mean_age_actors_actress'}, inplace=True)

#change type colone 'mean_age_actress'
#age_mean_year = age_mean_year.astype({'mean_age_actors_actress': int})

#check df age_mean_year
age_mean_year.head()

,startYear,mean_age_actors_actress
0,1918,36.56
1,1919,36.48
2,1920,36.91
3,1921,37.18
4,1922,37.29


## Table actor et table actress

In [293]:
# Actors
actors = actors_title_movies[actors_title_movies['category'] == "actor"]

#Actress
actress = actors_title_movies[actors_title_movies['category'] == "actress"]


## Groupby startYear et moyenne des ages des acteurs masculins

In [348]:
# calcul moyenne age groupby(by=["originalTitle" & "startYear"])
age_mean_year_actors = round(actors.groupby(by=["startYear"]).mean('age_actor_actress'),2)

#reset index
age_mean_year_actors = age_mean_year_actors.reset_index()

#drop colonnes
age_mean_year_actors.drop(columns=['index', 'birthYear'], inplace =True)

#rename nom colonne age_actor_actress
age_mean_year_actors.rename(columns={'age_actor_actress':'mean_age_actors'}, inplace=True)

#change type colone 'mean_age_actress'
#age_mean_year_actors = age_mean_year_actors.astype({'mean_age_actors': int})

#check df age_mean_year_actors
age_mean_year_actors.head()

,startYear,mean_age_actors
0,1918,39.71
1,1919,39.95
2,1920,40.11
3,1921,39.74
4,1922,40.31


## Groupby startYear et moyenne des ages des actrices

In [349]:
# calcul moyenne age groupby(by=["originalTitle" & "startYear"])
age_mean_year_actress = round(actress.groupby(by=["startYear"]).mean('age_actor_actress'),2)

#reset index
age_mean_year_actress = age_mean_year_actress.reset_index()

#drop colonnes
age_mean_year_actress.drop(columns=['index', 'birthYear'], inplace =True)

#rename nom colonne age_actor_actress
age_mean_year_actress.rename(columns={'age_actor_actress':'mean_age_actress'}, inplace=True)

#change type colone 'mean_age_actress'
#age_mean_year_actress = age_mean_year_actress.astype({'mean_age_actress': int})

#check df age_mean_year_actress
age_mean_year_actress.head()

,startYear,mean_age_actress
0,1918,31.30
1,1919,30.30
2,1920,30.76
3,1921,32.94
4,1922,31.97


## Creation df pour le graph

In [350]:
df_graph = age_mean_year.copy()
df_graph['mean_age_actress'] = age_mean_year_actress['mean_age_actress']
df_graph['mean_age_actors'] = age_mean_year_actors['mean_age_actors']
df_graph.head()

,startYear,mean_age_actors_actress,mean_age_actress,mean_age_actors
0,1918,36.56,31.30,39.71
1,1919,36.48,30.30,39.95
2,1920,36.91,30.76,40.11
3,1921,37.18,32.94,39.74
4,1922,37.29,31.97,40.31


## Le graph

In [351]:
fig = go.Figure() 

# mean actor and actress together per year
fig.add_trace(go.Scatter(x=df_graph.startYear, 
                    y=df_graph.mean_age_actors_actress, 
                    name="Actors and actress together",
                    line_shape='spline',
                    line_color='green'))

# mean only actress per year
fig.add_trace(go.Scatter(x=df_graph.startYear, 
                    y=df_graph.mean_age_actress,
                    name="Actress",
                    line_shape='spline',
                    line_color='rgb(231,107,243)'))

# mean only actor per year
fig.add_trace(go.Scatter(x=df_graph.startYear, 
                    y=df_graph.mean_age_actors,
                    name="Actors",
                    line_shape='spline',
                    line_color='blue'))


fig.update_layout(title ='<b>Mean age actors and actress since 1918<b>',
                    title_x = 0.5,
                    
                    width=1300,
                    height=600,
                    legend_title="Gender",
                    template='plotly_dark',
                    font_size=13
                    )

fig.show()

## Groupeby originalTitle et moyenne des ages des acteurs (tous genres confondus)
## Ensuite Groupeby startYear et moyenne des ages des acteurs (tous genres confondus)
(Si on ne procède pas ainsi nous n'avons plus les titres des films)

In [345]:
def groupbyinutile ():
    # calcul moyenne age groupby(by=["originalTitle" & "startYear"])
    age_mean = round(actors_title_movies.groupby(by=["originalTitle"]).mean('age_actor_actress'))

    #reset index
    age_mean = age_mean.reset_index()

    #Groupby startYear pour conserver le nom des films
    age_mean2 = age_mean.groupby('startYear').mean('age_actor_actress')

    #reset index
    age_mean2 = age_mean.reset_index()
    age_mean2.drop(columns=['index', 'birthYear'], inplace =True)

    #check df age_mean2
    age_mean2.head()


,level_0,originalTitle,startYear,age_actor_actress
0,0,#1 Serial Killer,2013.0,31.0
1,1,"#73, Shaanthi Nivaasa",2007.0,45.0
2,2,#BerhentiDiKamu,2021.0,69.0
3,3,"#Bitch, les filles et la violence",2015.0,50.0
4,4,#BlackSkin,2021.0,49.0


## Test du code

In [286]:
#film Easter Parade pour checker ensuite la moyenne manuellement
check_film = actors_title_movies[actors_title_movies['originalTitle'] == "Easter Parade"]
check_film

#Contrôle manuel = (49 + 26 + 25 + 25) /4 = 31

,index,primaryName,birthYear,deathYear,category,originalTitle,startYear,age_actor_actress
122,122,Fred Astaire,1899,1987,actor,Easter Parade,1948,49
123,123,Judy Garland,1922,1969,actress,Easter Parade,1948,26
124,124,Peter Lawford,1923,1984,actor,Easter Parade,1948,25
125,125,Ann Miller,1923,2004,actress,Easter Parade,1948,25


In [284]:
# selection du film : Easter Parade	et voir si la moyenne age_actor_actress est la même que ci-dessus

age_mean_film_Easter = age_mean2[age_mean2['originalTitle'] == "Easter Parade"]
age_mean_film_Easter

,originalTitle,startYear,age_actor_actress
45948,Easter Parade,1948.0,31.0
